#### Урок 3. Embedding word2vec fasttext
Задача поиск похожих по эмбеддингам

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0

!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

рабочие ссылки с твитами

https://disk.yandex.ru/i/v5HM-ENiGXZVpQ

https://disk.yandex.ru/i/koR5eMCToCZS2Q

В связи с удалением даных можно взять другой датасет, к примеру

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

##### пример работы с ним 

from corus import load_ods_gazeta

path = 'gazeta.csv.gz'

records = load_ods_gazeta(path)

next(records)

In [1]:
#!bzip2 -d lenta-ru-news.csv.bz2

In [2]:
import pandas as pd
import numpy as np

In [3]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec
from gensim.models.fasttext import FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import tqdm

In [4]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [5]:
assert True

#### что надо сделать:
1. объединить в одну выборку
2. на основе word2vec/fasttext слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)
3. Проверить насколько хорошо работают подходы

In [6]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive._append(negative).sample(frac=1)

In [7]:
df.tail()

,text,label
40674,"Слушаю сборник Axel Rudi Pell, 2011 год, ""The ...",positive
13107,"Я конечно потом исправилась, но это опять подт...",negative
94312,Сейчас я пишу проверку на циклы в графах. А чт...,positive
44556,"Девки не пускают меня домой, надо как то съеба...",positive
856,"каждый новый день похож на предыдущий, только ...",negative


In [8]:
list_df = df.text.to_list()
list_df[:2]

['@fivemiiles @shyrochka97 едьте! еще десять минут записи)',
 '@placeboonelove да ты оффигела)))).....человек заказывает....']

In [9]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

for line in list_df:
    spls = preprocess_txt(line)
    sentences.append(spls)
    c += 1
    if c > 1000:
        break

In [10]:
sentences = [i for i in sentences if len(i) > 2]
sentences[0][:10]

['fivemiiles', 'shyrochka97', 'едить', 'минута', 'запись']

In [11]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=2)

In [12]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for question in sentences:
    n_w2v = 0
    vector_w2v = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    w2v_index.add_item(counter, vector_w2v)
        
    counter += 1
    
    if counter > 1000:
        break

w2v_index.build(10)

True

In [27]:
def get_response(question, index, model, index_map, num_question):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, num_question, )
    return [list_df[i] for i in answers]

In [30]:
TEXT = "Новый год"
get_response(TEXT, w2v_index, modelW2V, index_map, 5)

['понедельник, 9 декабря\nне смотря что мама попыталась испортить с утра настроение, оно все равно супер)',
 'о это чувство блять\n:(\nбудто я виновата что мы познакомились блять\nбудто  я вообще сделала что - то плохое!\nпочему такое отношение',
 '(( Привет стресс и депрессия (( Как давно я вас не видел(((',
 'RT @nityluqitic: знаешь ли ты, вдоль ночных дорог, шла босиком, девочка-осьминог о_О',
 'перепутать расписание 8-ого класса с 11-ым .Настя это умеет.:D:D:D']

In [32]:
TEXT = "Боль"
get_response(TEXT, w2v_index, modelW2V, index_map, 5)

['У сегодняшнего дня были все составляющие, чтобы день был хорошим, а ощущения хорошести нет :(',
 '@sountone Я помню ее!"Просыпайся, просыпайся, просыпайся друг!\nпосмотри, посмотри, посмотри вокруг!" )',
 'RT @fm_mode: "ёлочная игрушка" в Ницце ) #flk #foto http://t.co/SkfWZk584G',
 '@Ann23_06 У меня мама думает что я накурился :DDDDDD',
 'Кста, мне показалось или морозоапокалипсиса всё таки пока не предвидится?)']

для новостных статей задание такое же
1. на основе word2vec/fasttext слоя Embedding реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

In [17]:
list_news = pd.read_csv('lenta-ru-news.csv').text.to_list()
list_news[:2]

/var/folders/q4/vb5sklnx7px01y0x6wz65s0m0000gn/T/ipykernel_4710/3150361606.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  list_news = pd.read_csv('lenta-ru-news.csv').text.to_list()


['Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. \n«Русский инвалид», 16 сентября 1914 года.',
 'Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соот

In [18]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

for line in list_news:
    spls = preprocess_txt(line)
    sentences.append(spls)
    c += 1
    if c > 1000:
        break

In [19]:
sentences = [i for i in sentences if len(i) > 2]
sentences[0][:10]

['бой',
 'сопоцкина',
 'друскеник',
 'закончиться',
 'отступление',
 'германец',
 'неприятель',
 'приблизиться',
 'север',
 'осовца']

In [20]:
%%time
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=2)

CPU times: user 1.2 s, sys: 11.9 ms, total: 1.21 s
Wall time: 481 ms


In [21]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for question in sentences:
    n_w2v = 0
    vector_w2v = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    w2v_index.add_item(counter, vector_w2v)
        
    counter += 1
    
    if counter > 1000:
        break

w2v_index.build(10)

True

In [23]:
TEXT = "Преступление"
get_response(TEXT, w2v_index, modelW2V, index_map, 5)

['Шесть членов русскоязычной банды в Нью-Йорке, которая проходит вматериалах дела под названием "Бригада Гуфельда-Куценко",признали себя виновными в вымогательстве, похищении людей,грабежах и поджогах. Как отмечается в распространенном впонедельник заявлении прокуратуры Восточного округа Нью-Йорка,пойдя на это, обвиняемые рассчитывают на снисхождение судьи. Засовершенные преступления им всем грозит пожизненное заключение. Заседание суда, на котором им будут определены сроки наказания,должно состояться в ближайшие три месяца. Члены "бригады" - главарь Дмитрий Гуфельд, АлександрКуценко, Игорь Котов, Андрей Якубов, Александр Кройтор и ЛоренсоМуссо - орудовали в нью-йоркских районах Brooklyn и Queens, гдепроживают большие русскоязычные общины. Банда занималась рэкетом,захватом заложников с целью получения выкупа, а также cбороминформации, например, о системе охраны ювелирного магазина. Бандиты признались, что подожгли в Queens восемьмагазинов, пытаясь запугать их владельцев. Кстати, притуш

In [24]:
TEXT = "Москва"
get_response(TEXT, w2v_index, modelW2V, index_map, 5)

['В сети Интернет открылся персональный сайт мэра Москвы Юрия Лужкова. На сайте размещена биография мэра, в разделе "В политике и на работе" изложена позиция Лужкова по наиболее острым проблемам (выборы президента, демократия, Косово, национализм, экономическая политика в Москве и др.). Ежедневно обновляемый раздел "Новости" представляет материалы по деятельности Лужкова как  мэра столицы и как политического деятеля. На сайте имеется дайджест материалов СМИ, посвященных мэру и блоку "Отечество - Вся Россия", рассказывается также об увлечениях Лужкова и его семье. В разделе "Обратная связь" московский мэр отвечает на самые животрепещущие вопросы посетителей сайта. В частности, оказывается, что Лужкову вовсе не наплевать на творчество Пола Маккартни, что мэр любит фильм "Чапаев" и настоящий свекольный борщ со сметаной, а также ощущает себя музыкальным инструментом органом. 16 сентября в 17 часов состоится интернет-конференция Юрия Лужкова. Мероприятие организовано при содействии компании

In [25]:
TEXT = "Война"
get_response(TEXT, w2v_index, modelW2V, index_map, 5)

['\nВ пятницу французская газета Le Monde публикует интервью с президентом Чечни Асланом Масхадовым. Как подчеркивает бравшая интервью журналистка Софи Шиаб, прошло уже более года с тех пор, как Масхадов последний раз давал такие подробные ответы о своей позиции по российско-чеченским отношениям. Мы приводим русский перевод этого интервью. Чем вы отвечаете русским, которые обвиняют ваших "радикалов" в организации терактов? В этих терактах виновны сами русские. Для того, чтобы устроить такой взрыв, нужна огромная сумма денег в самой России. Настоящий чеченец, даже если он очень хочет отомстить, не может разрушить жилой дом. Мы этого не делали даже во время войны. Нынешняя ситуация все больше напоминает 1994 год. В Кремле опять проблемы - опять приближаются выборы. Они еще не нашли наследника, который мог бы честно победить, и они не могут допустить к власти нового человека, который раскроет их преступления. Каково же выбранное ими решение? Война или чрезвычайное положение. И наиболее по